# Sprawdzenie nauczonego modelu

## Użyte biblioteki

In [28]:
import os
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Lambda
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
import cv2

## Zmienne globalne

In [29]:
CHECK_PATH = 'check'

FIRST_PHOTO = os.path.join(CHECK_PATH, "first.jpg")
SECOND_PHOTO = os.path.join(CHECK_PATH, "second.jpg")


## Utworzenie folderu do sprawdzenia

In [11]:
!mkdir $CHECK_PATH

## Definicja modelu

In [30]:
# Definiowanie kształtu wejścia
input_shape = (250, 250, 3)

# Definiowanie pod-sieci (ang. base network)
def create_base_network(input_shape):
  input = Input(shape=input_shape)
  x = Conv2D(32, (5, 5), activation='relu')(input)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu')(x)
  x = MaxPooling2D(pool_size=(2, 2))(x)
  x = Flatten()(x)
  x = Dense(256, activation='relu')(x)
  return Model(inputs=input, outputs=x)

# Tworzenie pod-sieci
base_network = create_base_network(input_shape)

# Definiowanie wejść dla siamskiej sieci neuronowej
input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# Uzyskiwanie wektorów cech z pod-sieci
processed_a = base_network(input_a)
processed_b = base_network(input_b)

# Funkcja Lambda do obliczenia odległości Euklidesowej między wektorami cech
def euclidean_distance(vects):
  x, y = vects
  return tf.sqrt(tf.reduce_sum(tf.square(x - y), axis=1, keepdims=True))

distance = Lambda(euclidean_distance, output_shape=(1,))([processed_a, processed_b])

# Definiowanie modelu siamskiej sieci neuronowej
model = Model([input_a, input_b], distance)

# Kompilowanie modelu
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Wczytanie wag nauczonego modelu
model.load_weights('model/model_1.h5')


## Przetworzenie obrazów wejściowych
Najlepiej je wziąć z folderu extracted

In [31]:
# Funkcja do wczytania i przetworzenia obrazu
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (250, 250))  # Dostosuj rozmiar do wymagań modelu
    img = img.astype('float32') / 255.0  # Normalizacja
    return img

# Wczytanie i przetworzenie obu zdjęć
img1_path = FIRST_PHOTO
img2_path = SECOND_PHOTO

img1 = preprocess_image(img1_path)
img2 = preprocess_image(img2_path)
                        
# plt.imshow(img1)

# Utworzenie pary obrazów
img1 = np.expand_dims(img1, axis=0)  # Dodanie wymiaru partii
img2 = np.expand_dims(img2, axis=0)  # Dodanie wymiaru partii
pair = [img1, img2]


## Predykcja

In [32]:
# Przewidywanie za pomocą modelu
prediction = model.predict(pair)

# Funkcja do oceny podobieństwa
def is_same_person(prediction, threshold=0.5):
    return prediction < threshold

# Ocena podobieństwa
result = is_same_person(prediction)
print(result[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
True
